- Small-World + Modular Netzwerk hinzufuegen, das beides vereint

- Evaluationsmetriken erstmal auslesen

- Modular mit mehr communities testen

- More Layers

- RNNs



Questions that need Clarification:
- Why do Episode lengths differ between topologies?
- What kind of baseline does Fully Connected provide? Does it have inherent advantages because of more connections or even more parameters?
- Influence of where and which are input and output?
- What role does the number of hidden nodes play?
- What does input stubs mean?



TODOs:
- understand exact NEAT generation process


### 1   Big picture 
| Phase                          | Core research goal                                                         | Concrete deliverables you’ve listed                                                                                                                                                                                                                                                   | Extra choices you’ll need to lock-in                                                                                                                                                                  |
| ------------------------------ | -------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **1. Topology ⇢ RL fitness**   | Quantify how structural **form** affects one-shot RL performance           | – Implement FC, random-sparse, Watts-Strogatz SW, explicit modular, SW + modular (NEAT or direct generators)  <br>– Compute structural metrics (ρ, C, L, σ, Q, etc.)  <br>– Reward curves, sample-efficiency, parameter count  <br>– Correlation / ANOVA linking structure ↔ RL score | • Which RL tasks? (simple control vs. Atari vs. MuJoCo)  <br>• Which RL algorithm family (value-based, actor-critic, evolutionary)?  <br>• How many random seeds for significance?                    |
| **2. Continual / Lifelong**    | Identify structures that **retain & transfer** across a task sequence      | – Define task curriculum (e.g., Continual-World, MiniGrid-LevelGen, ProcGen variations)  <br>– Metrics:  **Forward transfer**, **Average forgetting**, final **area under reward curve**  <br>– Compare topologies from Phase 1 under identical curricula                             | • Curriculum length & similarity gradient  <br>• Whether to freeze weights vs. fine-tune  <br>• Use regularisation baselines (EWC, SI, L2) as controls                                                |
| **3. NDP-controlled rigidity** | Let a **Neural Developmental Program** decide what stays rigid vs. plastic | – Implement/core-use Nisioti-style NDP that outputs connection graph **and** a “rigidity gate” per edge/neuron  <br>– Train NDP over the same curricula; measure emergent rigidity patterns  <br>– Analyse correlations between learned rigidity and metrics from Phases 1–2          | • Encoding of rigidity (scalar temperature? binary mask? synaptic consolidation coefficient)  <br>• How often NDP can mutate structure during life  <br>• Compute cost of plasticity vs. frozen parts |



### 2 Experimental stack

| Layer                  | Recommended option                                                                                                                                                                                                                                             | Rationale                                                                                                             |
| ---------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------- |
| **RL environments**    | **Start small**: CartPole, MountainCarContinuous, LunarLander, then **scale** to *MiniGrid LevelGen*, *Continual-World (HalfCheetah angles)*, or *ProcGen* variants.                                                                                           | Cheap early iterations; later tasks give you continuous control and vision-based curricula.                           |
| **RL algorithms**      | • PPO (stable, on-policy)<br>• A2C (fast baseline)<br>• ES/NEAT (for graph-evolving baselines)                                                                                                                                                                 | Gives you gradient-based + evolutionary flavours, and both can share the same graph substrate.                        |
| **Graph generators**   | • **FC**: trivial dense adjacency<br>• **Random-sparse**: Erdős–Rényi *(N, p)*<br>• **Modular**: Stochastic Block Model *(p\_intra ≫ p\_inter)*<br>• **SW**: Watts-Strogatz *(k, β)*<br>• **SW+Mod**: SBMs with WS inside blocks + random long-range shortcuts | All have two or three tunable knobs you can sweep while “staying inside” the topology class (as we detailed earlier). |
| **Structural metrics** | ρ, C, L, σ, Q, assortativity, average degree, weighted efficiency                                                                                                                                                                    | Store per-run; you’ll need them for correlation & ANOVA.                                                              |



### 3 Statistics & analysis blueprint

1. **Phase 1 cross-section:**
   *Two-way ANOVA* with factors **Topology** × **Task** on (a) final episodic reward, (b) AUC of learning curve, (c) parameter count.
   Follow with **Pearson/Spearman correlation** between each structural metric and each performance metric.

2. **Phase 2 continual:**
   Compute for every task *t* in the curriculum:

   $$
   \text{Forgetting}(t)=\max_{k\le t} R_k - R_t
   $$

   then report **Average Forgetting**, **Forward Transfer** (first-episode reward on task *t+1*), and **Backward Transfer**.
   Same ANOVA structure but substitute *Topology* × *Curriculum*.

3. **Phase 3 rigidity patterns:**
   *Cluster* learned rigidity coefficients and relate cluster membership to (a) module boundaries, (b) node centrality, (c) structural metric values.
   Possible tools: Mantel test (matrix correlation between rigidity matrix and adjacency), mutual information between rigidity mask and community labels.


### 4 Open design questions (worth nailing down early)

| Question                                  | Why it matters                                                                        | Typical choices                                                                      |
| ----------------------------------------- | ------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------ |
| **What counts as “good” RL performance?** | Sample-efficiency vs. asymptotic vs. compute budget can lead to opposite conclusions. | Use both AUC (early learning) and final reward; normalise by FLOPs or wall-clock.    |
| **Weight-sharing vs. per-edge weights?**  | Graphs with repeated edge weights behave differently (esp. modular).                  | Probably unique weights; but you can test hash-based parameter tying as an ablation. |
| **Plasticity rule during lifetime?**      | Hebbian, Oja, or gradient? For NDP you might inject online Hebbian updates.           | Start with SGD/Adam only; later add Hebbian channels if time allows.                 |
| **Plasticity cost**                       | Free plasticity biases toward fully plastic networks.                                 | Impose an L1 penalty on non-rigid gates or limit number of plastic edges.            |
| **Computation budget**                    | Determines number of random seeds × tasks × topologies you can afford.                | Rough planning: 5 seeds × 5 topologies × 4 tasks ≈ 100 training runs per phase.      |

- Trying to extract the effect of weight vs. the degree of the node
    - Apparently the others already tried to see if there is a difference in weight updates, depending on topology... In FC networks weight updates are quite similar throughout the network
- Metric about effect weights in non-fully connected networks 